## Objective
Develop a LangChain powered agent that integrates real-time utilities for current date and time. following capabilities are included:



1.   Automatically detect when a user query involves current date or time.
2.   Call the appropriate tool (get_current_date or get_current_time).
3.   Leverage LangChain’s tool-calling framework to handle reasoning and orchestration
4.   Maintain a reasoning trace using agent_scratchpad for clarity.
5.   Provide responses in a conversational and context-aware format, ensuring outputs are both precise and user-friendly.

In [1]:
!pip install langchain.openai
!pip install langchain_classic

In [ ]:
# import libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
#from langchain.memory import ConversationBufferMemory
from langchain_classic.memory import ConversationBufferMemory

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
# call the model
llm = ChatOpenAI(model="gpt-4o", api_key=openai_api_key)

In [ ]:
# define memory to maintain context throughout human interaction
memory = ConversationBufferMemory(
    # key under which conversation history will be stored
    memory_key="user_chat_history",
    return_messages=True
    )

In [ ]:
# create a prompt  from a list of messages. Each message is a tuple where the
#  first element is the message type (e.g., "system", "user", "placeholder") and
# the second element is the content
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assisteant. Use tools if necessary to answer user query. Since you are storing chat history, every time you generate a response, start by saying 'Referring to chat history' or 'Not referring to chat history' depending upon whether or not you are using chat history to generate a response and start the response on the next line"),
    # placeholder that will be filled with the conversation history. The variable_name="chat_history"
    # means that the memory component (defined earlier as memory) will inject the past conversation
    # turns into the prompt at this point.
    MessagesPlaceholder(variable_name="user_chat_history"),
    ("user", "{user_input}"),
    # The agent_scratchpad is where the agent internally keeps track of its thoughts,
    # the tools it decided to use, the observations it got back from tools, and its
    # planning steps before generating a final response.
    ("placeholder", "{agent_scratchpad}")
]
)

In [ ]:
# tool definition
from datetime import datetime, timezone
@tool
def get_utc_datetime():
  """ this function returns the current UTC date and time"""
  current_datetime = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")
  print(type(current_datetime))
  return f"Current UTC datetime is: {current_datetime}"
#get_utc_datetime()

tools = [get_utc_datetime]

In [ ]:
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# wrap the agent in executor
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

In [ ]:
response = agent_executor.invoke({"user_input": "explain machine learning in five lines"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...
Not referring to chat history

Machine learning is an area of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. Instead of being explicitly programmed to perform a task, these systems improve their performance through experience and data analysis. The core components of machine learning include algorithms, models, and training data. It can be used for various applications like classification, regression, clustering, and reinforcement learning. The ultimate goal is to enable systems to automatically learn and adapt without human intervention.

> Finished chain.
Not referring to chat history

Machine learning is an area of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. Instead of being explicitly programmed to perform a task, these systems improve their performance through experience and data analysis. The core components of 

In [ ]:
response = agent_executor.invoke({"user_input": "what is current time in Singapore"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `get_utc_datetime` with `{}`


<class 'str'>
Current UTC datetime is: 2026-01-23 10:03:33Referring to chat history

Singapore is 8 hours ahead of UTC. Given the current UTC time is 10:03:33, the current time in Singapore is 18:03:33.

> Finished chain.
Referring to chat history

Singapore is 8 hours ahead of UTC. Given the current UTC time is 10:03:33, the current time in Singapore is 18:03:33.


In [ ]:
response = agent_executor.invoke({"user_input": "my name is SB"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...
Not referring to chat history

Hello SB! How can I assist you today?

> Finished chain.
Not referring to chat history

Hello SB! How can I assist you today?


In [ ]:
response = agent_executor.invoke({"user_input": "where is Singapore located?"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...
Not referring to chat history

Singapore is a city-state and island country located in Southeast Asia. It lies at the southern tip of the Malay Peninsula, about 137 kilometers (85 miles) north of the equator. It is bordered by the Strait of Malacca to the west, the Riau Islands of Indonesia to the south, and the South China Sea to the east. Singapore is known for its strategic location as a major hub for international trade and travel.

> Finished chain.
Not referring to chat history

Singapore is a city-state and island country located in Southeast Asia. It lies at the southern tip of the Malay Peninsula, about 137 kilometers (85 miles) north of the equator. It is bordered by the Strait of Malacca to the west, the Riau Islands of Indonesia to the south, and the South China Sea to the east. Singapore is known for its strategic location as a major hub for international trade and travel.


In [ ]:
response = agent_executor.invoke({"user_input": "do you recall my name?"})
#print(response)
print(response["output"])



> Entering new AgentExecutor chain...
Referring to chat history

Yes, you mentioned that your name is SB. How can I assist you further, SB?

> Finished chain.
Referring to chat history

Yes, you mentioned that your name is SB. How can I assist you further, SB?


In [ ]:
# print chat history
print("Chat History")
for message in memory.load_memory_variables({}).get("user_chat_history"):
  print(message.content, "\n\n")

Chat History
explain machine learning in five lines 


Not referring to chat history

Machine learning is an area of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. Instead of being explicitly programmed to perform a task, these systems improve their performance through experience and data analysis. The core components of machine learning include algorithms, models, and training data. It can be used for various applications like classification, regression, clustering, and reinforcement learning. The ultimate goal is to enable systems to automatically learn and adapt without human intervention. 


what is current time in Singapore 


Referring to chat history

Singapore is 8 hours ahead of UTC. Given the current UTC time is 10:03:33, the current time in Singapore is 18:03:33. 


my name is SB 


Not referring to chat history

Hello SB! How can I assist you today? 


where is Singapore located? 


Not referring to chat histor